Genetic Algorithm
=======

# Theory

## Definition


## Examples


# Implementation


## Imports

In [2]:
import matplotlib
import numpy as np
import pandas as pd
from typing import Callable
import matplotlib.pyplot as plt
matplotlib.rcParams['animation.embed_limit'] = 2**128

## Parameters

## Algorithm


In [4]:
def GA(cost_func: Callable, get_neighborhood: Callable, number_of_offspring: int, x_0: float, generations: int = int(5e2)) -> tuple[float, list, int]:
    '''
    '''

    # algorithm data
    cost = [cost_func(x_0)]
    accepted_samples = 0
    x = [x_0]


    for i in range(generations):
        
        offsprings = ["List of solutions with mutation"] # [sample_offspring(x) for _ in range(number_of_offspring)]
        natural_selection = ["These off_springs compete with each other and survival of the fittest"]

        # place holder



## Results Visualization & Behaver Analysis


In [ ]:
x_optimal, accepted_samples, history = GA(cost_func=cost_func,
                                          sample_neighbor=get_neighbor,
                                          x_0=x_0, 
                                          cooling_schedule=(cooling_func, cooling_window), 
                                          )

In [42]:
i = 0
x = 1
while x > 0:
    x = 3 + np.random.normal()
    i += 1
x, i

(-0.6709468886932317, 37)